In [1]:
from ultralytics import YOLO

In [2]:
import cv2
import supervision as sv

In [3]:
import os

In [4]:
os.environ['USE_FLASH_ATTENTION'] = '1'

In [5]:
model = YOLO('yolov8x-worldv2.pt')

In [6]:
classes = ['white kilometer post', 
    'kilometer post', 
    'picket kilometer post', 
    'picket post', 
    'mile post', 
    'picket kilometer post with black numbers',  
    'big column', 
    'mechanism', 
    'big pillar', 
    'electric pole', 
    'pole with red and grey', 
    'thin long gray column',
    'picket kilometer post with white numbers on black plate',
    'mile marker with black digits',
    ""]

In [7]:
model.set_classes(classes)

In [8]:
model_path = '.'



VIDEO_PATH_1 = os.path.join(model_path, 'train-video.mp4')

In [9]:
VIDEO_PATH_2 = os.path.join(model_path, 'train-video-2.mp4')

In [10]:
VIDEO_PATH_FULL = os.path.join(model_path, 'train-video-full.mp4')

In [16]:
VIDEO_PATH_FULL_2 = os.path.join(model_path, 'train-video-full-2.mp4')

In [11]:
import numpy as np
import random

In [12]:
import pandas as pd
detection_df = pd.DataFrame(columns=["frame_id", "x1", "y1", "x2", "y2", "confidence", "class_id", "class_name"])


In [19]:
def process_frame(frame: np.ndarray, frame_num) -> np.ndarray:
    # Assuming model.predict() outputs results compatible with Detections.from_ultralytics
    results = model.predict(source=frame, device="0", conf=0.3, iou=0.8, line_width=2, augment=True, half=True)
    detections = sv.Detections.from_ultralytics(results[0])
    
    # Filter detections based on class names to ignore
    ignore_classes = ['mechanism', 'big pillar', 'electric pole', 'pole with red and grey', 'thin long gray column', 'big column']
    
    # Initialize the bounding box annotator
    box_annotator = sv.BoundingBoxAnnotator()

    # Iterate over each detection tuple
    for det in detections:
        xyxy, _, confidence, class_id, _, data = det
        class_name = data['class_name']
        
        if class_name not in ignore_classes:
            # Annotate frame with valid detections
            
            print([
                xyxy[0], xyxy[1], xyxy[2], xyxy[3],
                confidence,
                class_id,
                class_name
            ])
            # Append the detection data to the DataFrame
            detection_df.loc[len(detection_df)] = [
                frame_num,
                xyxy[0], xyxy[1], xyxy[2], xyxy[3],
                confidence,
                class_id,
                class_name
            ]
    
    frame = box_annotator.annotate(scene=frame, detections=detections)
    return frame

sv.process_video(source_path=VIDEO_PATH_FULL, target_path=f"result_full_1.mp4", callback=process_frame)
detection_df.to_csv("detection_results_1.csv", index=False)
detection_df = pd.DataFrame(columns=["frame_id", "x1", "y1", "x2", "y2", "confidence", "class_id", "class_name"])
sv.process_video(source_path=VIDEO_PATH_FULL_2, target_path=f"result_full_2.mp4", callback=process_frame)
detection_df.to_csv("detection_results_2.csv", index=False)



0: 384x640 (no detections), 183.7ms
Speed: 3.5ms preprocess, 183.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict4

0: 384x640 (no detections), 16.3ms
Speed: 2.0ms preprocess, 16.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict4

0: 384x640 (no detections), 20.0ms
Speed: 1.0ms preprocess, 20.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict4

0: 384x640 (no detections), 19.7ms
Speed: 1.0ms preprocess, 19.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict4

0: 384x640 (no detections), 16.6ms
Speed: 1.1ms preprocess, 16.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\predict4

0: 384x640 (no detections), 12.6ms
Speed: 0.0ms preprocess, 12.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Results sa

KeyboardInterrupt: 